In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import pickle
import cv2
import ast
import glob

from shutil import copyfile
import sys
from sklearn.model_selection import StratifiedKFold
sys.path.append('../input/tensorflow-great-barrier-reef')

# Set up paths

In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
HOME_DIR = '/kaggle/working/'
DATASET_PATH = 'COTS-YOLOv5-StratifiedKFold'
LABEL_DIR = '/kaggle/labels' # to save yolo converted labels
FOLD = 4 # number of folds to train

In [ ]:
!mkdir {HOME_DIR}{DATASET_PATH}
!mkdir {HOME_DIR}{DATASET_PATH}/images
!mkdir {HOME_DIR}{DATASET_PATH}/images/train
!mkdir {HOME_DIR}{DATASET_PATH}/images/val
!mkdir {HOME_DIR}{DATASET_PATH}/labels
!mkdir {HOME_DIR}{DATASET_PATH}/labels/train
!mkdir {HOME_DIR}{DATASET_PATH}/labels/val
!mkdir {LABEL_DIR}

# Helper Functions

In [ ]:
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    row['label_path'] = f'{LABEL_DIR}/video_{row.video_id}_{row.video_frame}.txt'
    return row

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

In [ ]:
def coco2yolo(image_height, image_width, bboxes):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

# Modify train dataframe

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df = df.progress_apply(get_path, axis=1) # add image path to dataframe
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x)) # str to list
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
df = df.query("num_bbox>0") # take only rows which contains bounding boxes
df['bboxes'] = df.annotations.progress_apply(get_bbox) # add number of bboxes column
# image resolution
df['width']  = 1280
df['height'] = 720
display(df.head(5))

# StratifiedKFold

In [ ]:
kf = StratifiedKFold(n_splits = 5)
df = df.reset_index(drop=True)
df['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(kf.split(df, y=df.video_id.tolist(), groups=df.sequence)):
    df.loc[val_idx, 'fold'] = fold
    
display(df.fold.value_counts())   
df.head(5)

# Save images

In [ ]:
for i in tqdm(range(len(df))):
    
    row = df.loc[i]
    
    if row.fold != FOLD: # train
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/images/train/{row.image_id}.jpg')
    
    elif row.fold == FOLD: # val
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/images/val/{row.image_id}.jpg')

# Convert YOLO annotation format

In [ ]:
cnt = 0
all_bboxes = []
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = [0]*num_bbox
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        bboxes_yolo  = coco2yolo(image_height, image_width, bboxes_coco)
        bboxes_yolo  = np.clip(bboxes_yolo, 0, 1)
        all_bboxes.extend(bboxes_yolo)
        for bbox_idx in range(len(bboxes_yolo)):
            annot = [str(labels[bbox_idx])]+ list(bboxes_yolo[bbox_idx].astype(str))+(['\n'] if num_bbox!=(bbox_idx+1) else [''])
            annot = ' '.join(annot)
            annot = annot.strip(' ')
            f.write(annot)
print('Missing:',cnt)

# Save annotations

In [ ]:
for i in tqdm(range(len(df))):
    
    row = df.loc[i]
    
    if row.fold != FOLD: # train
        copyfile(f'{row.label_path}', f'{HOME_DIR}{DATASET_PATH}/labels/train/{row.image_id}.txt')
    
    elif row.fold == FOLD: # val
        copyfile(f'{row.label_path}', f'{HOME_DIR}{DATASET_PATH}/labels/val/{row.image_id}.txt')